## Similarity Exercise

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

from sentence_transformers import SentenceTransformer
import faiss

In this exercise, you've been provided the title and abstract of 500 recent machine learning research papers posted on arXiv.org.

In [2]:
articles = pd.read_csv('../data/arxiv_papers.csv')
articles.head()

,title,abstract,url
0,GoT-R1: Unleashing Reasoning Capability of MLL...,Visual generation models have made remarkable ...,http://arxiv.org/abs/2505.17022v1
1,Delving into RL for Image Generation with CoT:...,Recent advancements underscore the significant...,http://arxiv.org/abs/2505.17017v1
2,Interactive Post-Training for Vision-Language-...,"We introduce RIPT-VLA, a simple and scalable r...",http://arxiv.org/abs/2505.17016v1
3,When Are Concepts Erased From Diffusion Models?,"Concept erasure, the ability to selectively pr...",http://arxiv.org/abs/2505.17013v1
4,Understanding Prompt Tuning and In-Context Lea...,Prompting is one of the main ways to adapt a p...,http://arxiv.org/abs/2505.17010v1


In [3]:
i = 0
print(f'Title: {articles.loc[i,"title"]}\n')
print(f'Text: {articles.loc[i,"abstract"]}')

Title: GoT-R1: Unleashing Reasoning Capability of MLLM for Visual Generation with Reinforcement Learning

Text: Visual generation models have made remarkable progress in creating realistic
images from text prompts, yet struggle with complex prompts that specify
multiple objects with precise spatial relationships and attributes. Effective
handling of such prompts requires explicit reasoning about the semantic content
and spatial layout. We present GoT-R1, a framework that applies reinforcement
learning to enhance semantic-spatial reasoning in visual generation. Building
upon the Generation Chain-of-Thought approach, GoT-R1 enables models to
autonomously discover effective reasoning strategies beyond predefined
templates through carefully designed reinforcement learning. To achieve this,
we propose a dual-stage multi-dimensional reward framework that leverages MLLMs
to evaluate both the reasoning process and final output, enabling effective
supervision across the entire generation pipeli

Let's try out a variety of ways of vectorizing and searching for semantically-similar papers.

### Method 1: Bag of Words

Fit a CountVectorizer to the abstracts of the articles with all of the defaults.  Then vectorize the dataset using the fit vectorizer. 

In [4]:
vectorizer = CountVectorizer()
vectorizer.fit(articles['abstract'])
X = vectorizer.transform(articles['abstract'])

print("Feature names:", vectorizer.get_feature_names_out())

Feature names: ['00' '000' '0009' ... 'zoom' 'zurich' 'zzaudio']


**Question:** How many dimensions do the embeddings have?

In [5]:
num_dimensions = len(vectorizer.get_feature_names_out())
print("Number of dimensions (vocabulary size):", num_dimensions)


Number of dimensions (vocabulary size): 7978


Now, let's use the embeddings to look for similar articles to a search query.

Apply the vectorizer you fit earlier to this query string to get an embedding. 

**Hint:** You can't pass a string to a vectorizer, but you can pass a list containing a string.

In [6]:
query = "vector databases for retrieval augmented generation"

query_vector = vectorizer.transform([query])

print(query_vector)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 6 stored elements and shape (1, 7978)>
  Coords	Values
  (0, 767)	1
  (0, 1851)	1
  (0, 2958)	1
  (0, 3139)	1
  (0, 6176)	1
  (0, 7723)	1


Now, we need to find the similarity between our query embedding and each vectorized article.

For this, you can use the [cosine similarity function from scikit-learn.](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html)

Calculate the similarity between the query embedding and each article embedding and save the result to a variable named `similarity_scores`.

In [7]:
similarity_scores = cosine_similarity(query_vector, X).flatten()
print("Similarity scores:", similarity_scores)

Similarity scores: [0.11020775 0.08461622 0.01780047 0.04145133 0.03806935 0.0814463
 0.06593805 0.03782347 0.         0.07059312 0.03462717 0.07968191
 0.03857584 0.020646   0.03197647 0.01893206 0.04279605 0.03979361
 0.05059374 0.01539738 0.01800706 0.0184995  0.03533326 0.
 0.01879115 0.03510393 0.12309149 0.         0.04543109 0.09829464
 0.10193473 0.062177   0.02197935 0.11200358 0.05493732 0.04188539
 0.03503922 0.06100889 0.09866713 0.07699905 0.02475369 0.03209979
 0.05309942 0.11826248 0.         0.02207554 0.0402259  0.09707137
 0.06233787 0.06225728 0.05412659 0.06384424 0.         0.
 0.01944039 0.1214167  0.0285133  0.09924856 0.         0.
 0.         0.06944891 0.07385489 0.0285831  0.06463956 0.02733833
 0.03914801 0.0955637  0.0843274  0.02810497 0.04072315 0.05878964
 0.05236631 0.09014453 0.03219948 0.0549235  0.03083865 0.01631688
 0.05990423 0.05402118 0.01885115 0.01747141 0.0209427  0.18057878
 0.06085806 0.07938842 0.0417756  0.         0.05858749 0.05255883
 

Now, we need to find the most similar results. To help with this, we can use the [argsort function from numpy](https://numpy.org/doc/stable/reference/generated/numpy.argsort.html), which will give the indices sorted by value. 

Use the argsort function to find the indices of the 5 most similar articles. Inspect their titles and abstracts. **Warning:** argsort sorts from smallest to largest.

In [8]:
top_indices = similarity_scores.argsort()[::-1][:5]
top_titles = articles.iloc[top_indices]['title']

print("Top 5 article titles:")
for title in top_titles:
    print("-", title)

Top 5 article titles:
- MIRB: Mathematical Information Retrieval Benchmark
- SCENIR: Visual Semantic Clarity through Unsupervised Scene Graph Retrieval
- WikiDBGraph: Large-Scale Database Graph of Wikidata for Collaborative Learning
- Explaining Neural Networks with Reasons
- Scan, Materialize, Simulate: A Generalizable Framework for Physically Grounded Robot Planning


Try using a tfidf vectorizer. How do the results compare?

In [9]:
vectorizer = TfidfVectorizer()
vectorizer.fit(articles['abstract'])
X = vectorizer.transform(articles['abstract'])

print("Feature names:", vectorizer.get_feature_names_out())

Feature names: ['00' '000' '0009' ... 'zoom' 'zurich' 'zzaudio']


In [10]:
num_dimensions = len(vectorizer.get_feature_names_out())
print("Number of dimensions (vocabulary size):", num_dimensions)

Number of dimensions (vocabulary size): 7978


In [11]:
query = "vector databases for retrieval augmented generation"

query_vector = vectorizer.transform([query])

print(query_vector)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6 stored elements and shape (1, 7978)>
  Coords	Values
  (0, 767)	0.44599854389252197
  (0, 1851)	0.5463161557306228
  (0, 2958)	0.11415110381868501
  (0, 3139)	0.2920125130744977
  (0, 6176)	0.44599854389252197
  (0, 7723)	0.45321901260219255


In [12]:
similarity_scores = cosine_similarity(query_vector, X).flatten()
print("Similarity scores:", similarity_scores)

Similarity scores: [0.0738423  0.04643354 0.00149363 0.00482652 0.00400722 0.00899285
 0.00670455 0.00459842 0.         0.00654712 0.00385468 0.00813586
 0.00386085 0.00175744 0.00310723 0.00202475 0.00424814 0.00454675
 0.00527572 0.0015219  0.00147971 0.0021455  0.00422946 0.
 0.00165823 0.00353547 0.03332313 0.         0.00508913 0.01154996
 0.02387566 0.00695964 0.01503263 0.04062112 0.03875465 0.00498018
 0.02635126 0.00652603 0.01074876 0.00708426 0.0158414  0.0104277
 0.00536147 0.01322956 0.         0.00198226 0.00389958 0.0107171
 0.02698612 0.01525687 0.00511446 0.04136264 0.         0.
 0.00250439 0.01245716 0.0018523  0.00831975 0.         0.
 0.         0.00893754 0.02915913 0.00275528 0.02246997 0.00275477
 0.01618404 0.01022199 0.00800934 0.00273123 0.00332047 0.00603277
 0.00489083 0.0102186  0.01359262 0.00453633 0.00372204 0.00177125
 0.00674237 0.00607998 0.00209091 0.00168125 0.0020232  0.26498602
 0.01901256 0.00701427 0.01523753 0.         0.00694431 0.00592098
 0

In [13]:
top_indices = similarity_scores.argsort()[::-1][:5]
top_titles = articles.iloc[top_indices]['title']

print("Top 5 article titles:")
for title in top_titles:
    print("-", title)

Top 5 article titles:
- MIRB: Mathematical Information Retrieval Benchmark
- WikiDBGraph: Large-Scale Database Graph of Wikidata for Collaborative Learning
- SCENIR: Visual Semantic Clarity through Unsupervised Scene Graph Retrieval
- HDLxGraph: Bridging Large Language Models and HDL Repositories via HDL Graph Databases
- Explaining Neural Networks with Reasons


### Method 2: Using a Pretrained Embedding Model

Now, let's compare how we do using the [all-MiniLM-L6-v2 embedding model](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2).

This will create a 384-dimensional dense embedding of each sentence.

In [14]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [15]:
sentences = ["This is an example sentence", "Each sentence is converted"]

embeddings = embedder.encode(sentences)
print(embeddings)

[[ 6.76568821e-02  6.34959117e-02  4.87131439e-02  7.93049559e-02
   3.74480635e-02  2.65279901e-03  3.93749736e-02 -7.09847594e-03
   5.93614466e-02  3.15370448e-02  6.00980595e-02 -5.29052205e-02
   4.06067669e-02 -2.59308070e-02  2.98428256e-02  1.12692453e-03
   7.35148787e-02 -5.03818952e-02 -1.22386634e-01  2.37028133e-02
   2.97265332e-02  4.24768552e-02  2.56337691e-02  1.99518725e-03
  -5.69191128e-02 -2.71598194e-02 -3.29035521e-02  6.60248771e-02
   1.19007163e-01 -4.58791517e-02 -7.26214498e-02 -3.25840227e-02
   5.23413569e-02  4.50553037e-02  8.25298857e-03  3.67024094e-02
  -1.39415646e-02  6.53918535e-02 -2.64272671e-02  2.06397686e-04
  -1.36643434e-02 -3.62810977e-02 -1.95043795e-02 -2.89738216e-02
   3.94270122e-02 -8.84090662e-02  2.62427074e-03  1.36713488e-02
   4.83062565e-02 -3.11566107e-02 -1.17329188e-01 -5.11690341e-02
  -8.85288268e-02 -2.18962803e-02  1.42986607e-02  4.44167592e-02
  -1.34814996e-02  7.43392557e-02  2.66383160e-02 -1.98762398e-02
   1.79191

Use this new embedder to vectorize the abstracts and then find the most similar to the query. How do the results compare to the other methods?

**Warning:** Creating embeddings for all of the articles may take a while.

In [16]:
abstract_embeddings = embedder.encode(articles['abstract'].tolist())


In [17]:
query_embedding = embedder.encode([query])
similarity_scores = cosine_similarity(query_embedding, abstract_embeddings).flatten()
print("query embeddings", query_embedding)
print("Similarity scores:", similarity_scores)

query embeddings [[-3.84719446e-02 -4.03677672e-02 -5.39208725e-02  4.15326916e-02
  -1.29077118e-02  2.70722639e-02  3.42489257e-02 -4.00642268e-02
  -4.13630418e-02 -2.66212467e-02  2.50468273e-02 -3.20716086e-03
   8.27724412e-02 -4.28548418e-02 -6.03753850e-02  3.29218134e-02
   2.27026939e-02  7.41225034e-02  5.59455566e-02 -2.97653768e-02
  -3.94970849e-02  4.32495959e-02  1.92625634e-02 -2.07796842e-02
   3.18871737e-02  1.39019201e-02  3.14147137e-02 -4.15339582e-02
   7.32684806e-02 -5.43152578e-02  3.72031070e-02  1.11246176e-01
  -4.10853103e-02  8.41100961e-02 -6.84786811e-02  3.51947509e-02
  -8.79902691e-02  9.79631692e-02 -3.46180759e-02 -1.90933738e-02
  -6.27434403e-02  5.79465143e-02  2.31922474e-02  2.95088869e-02
   7.42464140e-02  3.73492949e-03 -1.03116602e-01 -8.04822892e-03
   1.59991551e-02  4.23684753e-02 -8.15261900e-02  5.80240181e-03
  -2.97683235e-02 -2.58239619e-02  1.33035379e-02  2.49278303e-02
  -5.13842748e-03 -4.85661477e-02 -3.61686014e-02 -4.509227

### FAISS

The [Faiss library](https://faiss.ai/index.html) is a library for efficient similarity search and clustering of dense vectors. It can be used to automate the process of finding the most similar abstracts.

If we want to use cosine similarity, we need to use the Inner Product. We also need to normalize our vectors so that they all have length 1.

Use the [normalize function](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.normalize.html) to normalize both the abstract vectors and the query vector.

In [18]:
abstract_embeddings = normalize(abstract_embeddings, norm='l2', axis=1)
query_embedding = normalize(query_embedding, norm='l2', axis=1) 

Now, create an [IndexFlatIP object](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes#summary-of-methods) that has dimensions equal to the dimensionality of your vectors. Then add your normalized abstract vectors.

Hint: You can mimic the example [here](https://github.com/facebookresearch/faiss/wiki/Getting-started#building-an-index-and-adding-the-vectors-to-it), but substitute in the IndexFlatIP class.

In [19]:
index = faiss.IndexFlatIP(abstract_embeddings.shape[1])
index.add(abstract_embeddings.astype(np.float32))

Finally, use the [search function](https://github.com/facebookresearch/faiss/wiki/Getting-started#searching) on your index object to find the 5 most similar articles.

In [22]:
k = 5
distances, indices = index.search(query_embedding.astype(np.float32), k)
print("Top 5 most similar article titles:")
for i in range(k):
    idx = indices[0][i]
    score = distances[0][i]
    print(f"{i+1}. {articles.iloc[idx]['title']}")

Top 5 most similar article titles:
1. MIRB: Mathematical Information Retrieval Benchmark
2. Attributing Response to Context: A Jensen-Shannon Divergence Driven Mechanistic Study of Context Attribution in Retrieval-Augmented Generation
3. HDLxGraph: Bridging Large Language Models and HDL Repositories via HDL Graph Databases
4. The Atlas of In-Context Learning: How Attention Heads Shape In-Context Retrieval Augmentation
5. Distance Adaptive Beam Search for Provably Accurate Graph-Based Nearest Neighbor Search
